# SC42x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드에 대해서 **한 줄**로 간단하게 요약해주세요. (세션 노트를 참고하여도 좋습니다.)<br/>
> **Tip : 아래 문제를 먼저 수행한 후 모델 학습 등 시간이 오래 걸리는 셀이 실행되는 동안 아래 내용을 작성하면 시간을 절약할 수 있습니다.**

**N421**
- Stopwords(불용어) : 분석 대상 말뭉치의 단어들 중 너무 자주 등장해 분석의 의미가 없다고 판단한 단어들의 집합
- Stemming과 Lemmatization : 말뭉치를 tokenizing할 때 stemming은 단어의 형태를 변형하지 않고 단어의 중심의미를 갖고 있는 앞부분(어간)만 잘라서 가져오는 것이고, lemmatization은 단어의 형태를 기본형으로 변환하여 가져오는 것이다.
- Bag-of-Words : 문장에 등장하는 단어의 횟수만을 고려해 문장을 벡터화하는 방식.
- TF-IDF : 문장을 벡터화할 때 잘 등장하지 않는 단어에 가중치를 두어 벡터화하는 방식.

**N422**
- Word2Vec : 주변에 있는 단어를 단서로 하여 단어를 벡터화하는 방법
- fastText : 단어를 벡터화할 때 단어의 구성요소까지 3-6글자 단위로 쪼개 벡터화하는 방법으로 Word2Vec의 OOV 문제를 어느정도 해소할 수 있다.

**N423**
- RNN : sequential한 데이터를 입력할 때, 지금까지 입력된 데이터를 고려하여 출력값을 리턴하는 신경망 모델.
- LSTM, GRU : RNN의 기울기 폭발, 소실 문제를 해소하기 위해 Gate를 추가한 RNN 기반 신경망 모델.
- Attention : 각 time-step 마다의 hidden vector를 decoding 시에 활용해 장기 의존성 문제를 해소한 방법.

# Part 2 : Fake/Real News Dataset

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다.<br/>
어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요.<br/>
이번 스프린트 챌린지에선 [Fake/Real News Dataset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)을 사용하여 배운 것들을 복습해보는 시간을 갖겠습니다.

**주의 : 모델의 성능을 최대한 끌어올리는 것이 아닌 모델 구동에 초점을 맞춰주세요.<br/>
모든 문제를 완료한 후에도 "시간이 남았다면" 정확도를 올리는 것에 도전하시는 것을 추천드립니다.**

In [68]:
# 코드 실행 전 seed를 지정하겠습니다.
import numpy as np
import pandas as pd
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

## 2.0 데이터셋을 불러옵니다.

- 위 캐글 링크에서 데이터셋을 받아 업로드 합니다.<br/>
(직접 업로드하게 되면 시간이 꽤 걸리므로 **drive_mount** 나 **kaggle 연동**하시는 것을 추천드립니다.)

- 'label' 열을 만들어 Fake = 1, True = 0 로 레이블링해줍니다.
- 두 파일을 합쳐 하나의 데이터프레임에 저장해 준 후 데이터를 섞어줍니다.

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
df_fake = pd.read_csv('/content/drive/MyDrive/Fake.csv')
df_true = pd.read_csv('/content/drive/MyDrive/True.csv')
df_fake['label'] = 0
df_true['label'] = 1

df_fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [71]:
df = pd.concat([df_fake, df_true])

# 행 섞기
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


## 2.1 TF-IDF 를 활용하여 특정 뉴스와 유사한 뉴스 검색하기

시간상 특별한 **전처리 없이** 아래 태스크를 수행하겠습니다.

### 2.1.1 TFidfVectorizer를 사용하여 문서-단어 행렬(Document-Term Matrix) 만들기

In [72]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [73]:
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [74]:
tfidf = TfidfVectorizer(stop_words='english', max_features=100)

dtm_tfidf = tfidf.fit_transform(df.text)

dtm_tfidf = pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf.head()

,000,2016,according,administration,america,american,americans,asked,called,campaign,china,city,clinton,committee,congress,country,court,day,democratic,democrats,department,did,don,donald,election,federal,foreign,friday,going,government,group,hillary,house,image,including,just,know,law,like,make,...,public,republican,republicans,reuters,right,russia,russian,said,say,saying,says,secretary,security,senate,state,statement,states,support,tax,think,thursday,time,told,trump,tuesday,twitter,united,vote,want,war,washington,way,wednesday,week,white,women,work,world,year,years
0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.515299,0.221246,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.232898,0.000000,0.000000,0.0,0.175822,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.420828,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.188896,0.000000,0.000000,0.000000,0.000000,0.0,0.234041,0.000000,0.000000,0.0
1,0.0,0.0,0.000000,0.183190,0.044892,0.000000,0.000000,0.000000,0.041490,0.041127,0.063003,0.0,0.000000,0.106183,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.031154,0.041156,0.000000,0.143978,0.047378,0.000000,0.000000,0.046065,0.000000,0.261336,0.000000,0.0,0.034636,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.048403,0.028469,0.042133,0.106736,0.000000,0.281215,0.000000,0.043482,0.000000,0.049311,0.447956,0.0,0.035270,0.044608,0.036522,0.0,0.0,0.000000,0.046922,0.000000,0.034431,0.365807,0.0000,0.0,0.038207,0.000000,0.0,0.0,0.075784,0.042245,0.046241,0.000000,0.240405,0.0,0.000000,0.045513,0.035526,0.0
2,0.0,0.0,0.000000,0.347947,0.000000,0.160179,0.000000,0.177877,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.19131,0.000000,0.000000,0.168594,0.000000,0.0,0.190746,0.000000,0.000000,0.000000,0.000000,0.185487,0.000000,0.000000,0.161175,0.141545,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.169726,0.0,0.000000,...,0.000000,0.000000,0.000000,0.108148,0.320107,0.000000,0.000000,0.493046,0.000000,0.165179,0.000000,0.187321,0.170168,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.106893,0.3466,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.175657,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,0.0,0.0,0.152444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.141880,0.000000,0.000000,0.000000,0.0,0.000000,0.141395,0.000000,0.113368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.154509,0.0,0.126035,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.194201,0.412155,0.314866,0.000000,0.158228,0.170005,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.129835,0.000000,0.511975,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.178402,0.000000,0.074386,0.294941,0.000000,0.0,0.082788,0.000000,0.00000,0.069904,0.000000,0.000000,0.080981,0.0,0.000000,0.000000,0.077884,0.055856,0.000000,0.000000,0.258134,0.084943,0.076069,0.000000,0.000000,0.082466,0.066935,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.216238,...,0.163236,0.417837,0.000000,0.051042,0.075540,0.000000,0.000000,0.116350,0.072082,0.000000,0.000000,0.088409,0.000000,0.0,0.063234,0.000000,0.065480,0.0,0.0,0.079628,0.084126,0.127939,0.000000,0.302698,0.0000,0.0,0.068501,0.174825,0.0,0.0,0.000000,0.000000,0.000000,0.074934,0.071836,0.0,0.000000,0.000000,0.000000,0.0


### 2.1.2 KNN 알고리즘을 사용하여 유사한 문서 검색하기

- **42번 인덱스의 문서**와 가장 유사한 **5개 문서(42번 포함)의 인덱스**와 **해당 인덱스의 레이블**을 나타내주세요.
- NN 모델의 파라미터 중 `algorithm = 'kd_tree'` 로 설정합니다.

In [75]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_tfidf)

NearestNeighbors(algorithm='kd_tree')

In [76]:
nearest_ind = nn.kneighbors([dtm_tfidf.iloc[42]])[1][0]

# 가장 유사한 문서 5개의 인덱스와 label
df.iloc[nearest_ind]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


,title,text,subject,date,label
33954,WATCH: AMAZING THING HAPPENED When H.S. Valedi...,A 70-year tradition at East Liverpool High Sch...,politics,"Jun 27, 2017",0
42,WATCH: AMAZING THING HAPPENED When H.S. Valedi...,A 70-year tradition at East Liverpool High Sch...,left-news,"Jun 27, 2017",0
36664,Heroic Mom Battling Cancer Caught On Camera R...,"Cancer can take a lot out of a person, especia...",News,"July 30, 2016",0
31854,Queen Elizabeth praises British spirit in Chri...,"SANDRINGHAM, England (Reuters) - Britain s Que...",worldnews,"December 25, 2017",1
5235,SWEDISH CITIZENS Get DISTURBING News About Lik...,When will government officials who allowed thi...,left-news,"Apr 13, 2017",0


## 2.2 Keras Embedding을 사용하여 분류하기

### 2.2.0 데이터셋 split

- Train, Test 데이터셋으로 분리(Split)하여 주세요.

In [77]:
# 이 곳에 답안을 작성하시길 바랍니다.
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

X = df.text
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [78]:
X_train.shape, X_test.shape

((35918,), (8980,))

### 2.2.1 단어 벡터의 평균을 이용하여 분류해보기

N422에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길고 시간이 오래 걸리므로 시간상 epoch 수를 **10 이하**로 하는 것을 추천드립니다.<br/>
모델 구동이 목적이므로 임베딩 차원 수를 크지 않게(50이하)로 설정해주세요.<br/>
**권장사항 : `max_len` 은 텍스트 길이 평균보다 높게 설정해주세요.**<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [88]:
# 이 곳에 답안을 작성하시길 바랍니다

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
print('평균 텍스트 길이 :', np.mean(list(map(len, X_train_tokenized))))
del(X_train_tokenized)

평균 텍스트 길이 : 297.85322122612615


In [89]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

125875

In [90]:
max_len = 300
def encoding(text):
  text_encoded = tokenizer.texts_to_sequences(text)
  text_paded = pad_sequences(text_encoded, maxlen=max_len)
  return text_paded
X_train_enc = encoding(X_train)
X_test_enc = encoding(X_test)

In [91]:
emb_dim = 50
model1 = Sequential()
model1.add(Embedding(10000, emb_dim, input_length=max_len)) # vocab_size가 너무 커서 10000으로 줄임
model1.add(GlobalAveragePooling1D())
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model1.fit(X_train_enc, np.array(y_train), batch_size = 32, epochs=10, validation_split=0.2)

Epoch 1/10
898/898 [==============================] - 7s 7ms/step - loss: 0.3605 - acc: 0.9036 - val_loss: 0.1803 - val_acc: 0.9645
Epoch 2/10
898/898 [==============================] - 6s 6ms/step - loss: 0.1310 - acc: 0.9724 - val_loss: 0.1089 - val_acc: 0.9776
Epoch 3/10
898/898 [==============================] - 6s 6ms/step - loss: 0.0856 - acc: 0.9813 - val_loss: 0.0811 - val_acc: 0.9827
Epoch 4/10
898/898 [==============================] - 6s 6ms/step - loss: 0.0643 - acc: 0.9855 - val_loss: 0.0665 - val_acc: 0.9859
Epoch 5/10
898/898 [==============================] - 6s 6ms/step - loss: 0.0509 - acc: 0.9881 - val_loss: 0.0569 - val_acc: 0.9868
Epoch 6/10
898/898 [==============================] - 6s 6ms/step - loss: 0.0417 - acc: 0.9905 - val_loss: 0.0495 - val_acc: 0.9883
Epoch 7/10
898/898 [==============================] - 6s 6ms/step - loss: 0.0344 - acc: 0.9925 - val_loss: 0.0454 - val_acc: 0.9884
Epoch 8/10
898/898 [==============================] - 6s 6ms/step - loss: 0.

In [92]:
loss1, acc1 = model1.evaluate(X_test_enc, y_test)
print('loss : {:.4f}, acc : {:.4f}'.format(loss1, acc1))

281/281 [==============================] - 1s 3ms/step - loss: 0.0311 - acc: 0.9919
loss : 0.0311, acc : 0.9919


### 2.2.2 LSTM을 사용하여 텍스트 분류 수행해보기

N423에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길어 시간이 매우 오래 걸리므로 <br/>
**층을 최소한으로 쌓고**, epoch 수를 **3 이하**로 하는 것을 추천드립니다.<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [84]:
# 이 곳에 답안을 작성하시길 바랍니다
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping

In [93]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(10000, emb_dim, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

model2 = RNN()
model2.summary()
model2.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model2.fit(X_train_enc, y_train, batch_size=32, epochs=3, validation_split=0.2)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 300, 50)           500000    
                                                                 
 lstm_2 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [94]:
loss2, acc2 = model2.evaluate(X_test_enc, y_test)
print('loss : {:.4f}, acc : {:.4f}'.format(loss2, acc2))

281/281 [==============================] - 6s 22ms/step - loss: 0.1465 - accuracy: 0.9719
loss : 0.1465, acc : 0.9719


### 2.2.3 위에서 실행한 내용에 대해 다시 알아봅시다.

#### a) 데이터셋을 학습할 때 사용하는 `pad_sequences`  메서드에 대해 설명해주세요.<br/>어떤 기능을 하나요? 모델을 학습할 때 왜 필요한가요?

*이곳에 답안을 입력해주세요*
각 sequence(문장, input)의 길이를 똑같이 맞춰준다. input으로 들어가는 데이터의 크기를 통일해주고, 지나치게 긴 문장을 잘라 연산 시간을 줄인다.

#### b) 2.2.1과 2.2.2에서 사용한 각 모델의 evaluation 성능은 어떻게 나왔나요?<br/>각 모델의 장단점은 무엇이라고 생각하나요?

*이곳에 답안을 입력해주세요*  
 임베딩벡터의 평균을 이용한 모델 99.19%, LSTM 모델이 97.19%로 전자의 모델 성능이 조금 더 높게 나왔다.  
 Embedding만 사용하는 경우 연산 속도가 빠르지만, 문장의 맥락이 고려되지 않기 때문에 맥락이 중요한 문제에서는 좋은 성능을 발휘하지 못할 것 같다.  
 LSTM의 경우 연산 속도는 느리지만 sequence의 맥락을 학습한다는 장점이 있다. (이번 문제의 경우 문장에 사용된 단어가 중요한 feature로 작용하지 않았을까 추측됨)

#### c) 종래의 RNN(Recurrent Neural Networks) 대신 LSTM(Long-Short Term Memory)을 사용하는 이유는 무엇인가요?<br/>(i.e. RNN에 비해 LSTM의 좋은 점을 설명해주세요.)

*이곳에 답안을 입력해주세요*
RNN의 기울기 소실, 폭발으로 인한 장기 의존성 문제를 LSTM은 cell-state와 기억의 정도를 조절하는 gate를 통해 해소하였다. 즉, LSTM은 RNN에 비해 긴 sequence 데이터도 잘 학습할 수 있다.

#### d) LSTM이나 RNN을 사용하는 예시를 **3개**이상 제시하고 해당되는 경우에 왜 LSTM이나 RNN을 사용하는 것 적절한지 간단하게 설명해주세요.

*이곳에 답안을 입력해주세요*
- 경우 : 감정 분석, 기계 번역, 주식 가격 예측
- 적절한 이유 : 각각의 문제가 sequential한 데이터의 맥락을 파악하는 것이 중요한 문제이기 때문이다.

#### e) 이외에 N424 에서 배운 자연어처리 모델과 관련된 키워드를 3개 이상 적어주세요. <br/> (해당 키워드에 대한 설명은 옵션입니다.)

*이곳에 답안을 입력해주세요*
- Transformer : Attention의 개념을 발전시키고 RNN 구조를 사용하지 않아 sequential한 데이터를 한번에 입력, 연산하여 학습 속도와 성능을 향상시킨 모델
- GPT, BERT : 둘 다 Transformer 구조를 변형하여 만들었으며, 사전 학습된 언어 모델이다. 각각 NLU와 NLG에 강점이 있다.

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- 2.1 에서 TF-IDF(`TfidfVectorizer`)가 아닌 방법을 사용하여 유사도 검색을 수행해보세요.<br/>
TF-IDF와 해당 방법의 차이를 설명해주세요. 
- 2.2 에서 사용한 방법을 재사용하되 하이퍼 파라미터를 조정하거나 모델 구조를 변경하여 성능을 올려봅시다.<br/>**(주의 : GridSearch, RandomSearch 등의 방법을 사용하여도 좋으나 시간이 오래 걸리므로 범위를 잘 선택해야 합니다.)**

In [95]:
# 이 곳에 답안을 작성하시길 바랍니다
# 2.2 embedding dimension, epochs 조절
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(10000, 100, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

model3 = RNN()
model3.summary()
model3.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model3.fit(X_train_enc, y_train, batch_size=32, epochs=5, validation_split=0.2)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding_6 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm_3 (LSTM)               (None, 64)                42240     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_6 (Activation)   (None, 256)               0         
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [96]:
loss3, acc3 = model3.evaluate(X_test_enc, y_test)
print('loss : {:.4f}, acc : {:.4f}'.format(loss3, acc3))

281/281 [==============================] - 7s 23ms/step - loss: 0.0345 - accuracy: 0.9879
loss : 0.0345, acc : 0.9879
